In [44]:
from language_identifier import LanguageIdentifier
import numpy as np
import pandas as pd

In [ ]:
LangId = LanguageIdentifier()

In [45]:
deu_pred = LangId.predict_probs(f"corpora/deu_tst.txt")
eng_pred = LangId.predict_probs(f"corpora/eng_tst.txt")
fra_pred = LangId.predict_probs(f"corpora/fra_tst.txt")
ita_pred = LangId.predict_probs(f"corpora/ita_tst.txt")
nld_pred = LangId.predict_probs(f"corpora/nld_tst.txt")
spa_pred = LangId.predict_probs(f"corpora/spa_tst.txt")

In [47]:
deu_true = np.full(len(deu_pred),'deu')
eng_true = np.full(len(eng_pred),'eng')
fra_true = np.full(len(fra_pred),'fra')
ita_true = np.full(len(ita_pred),'ita')
nld_true = np.full(len(nld_pred),'nld')
spa_true = np.full(len(spa_pred),'spa')

In [106]:
all_wrong = pd.DataFrame(columns = ["phrases", "pred"])

languages = ['deu', 'eng', 'fra', 'ita', 'nld', 'spa']
for language in languages:

    probs = pd.DataFrame(LangId.predict_probs(f"corpora/{language}_tst.txt"))
    probs['pred'] = probs.apply(lambda row: max(row, key=lambda x: x[1])[0], axis=1)
    probs["true"] = np.full(len(probs), language)
    probs_wrong = probs[probs['pred'] != probs['true']]

    with open(f'corpora/{language}_trn.txt', 'r', encoding='utf-8') as file:
        raw = file.read()
    lines = raw.split('\n')
    lines_data = []
    for index in probs_wrong.index:
        for line in lines:
            if line.startswith(str(index) + '\t'):
                lines_data.append(line)
    all_wrong = pd.concat([all_wrong, pd.DataFrame({"phrases": lines_data, "pred": probs_wrong["pred"].tolist(), 
                                                    "true": probs_wrong["true"].tolist()})], ignore_index = True)

In [108]:
all_wrong.to_csv("all_wrong.csv", index=False)